In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,32206
2,Huelva,Confirmados PDIA 14 días,1479
3,Huelva,Tasa PDIA 14 días,"288,20858584874406"
4,Huelva,Confirmados PDIA 7 días,630
5,Huelva,Total Confirmados,32386
6,Huelva,Curados,30014
7,Huelva,Fallecidos,386


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  32206.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8699.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 516 personas en los últimos 7 días 

Un positivo PCR cada 240 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,32206.0,630.0,1479.0,513170.0,122.766335,288.208586,100.0
Huelva-Costa,18552.0,382.0,826.0,289548.0,131.929766,285.272217,65.0
Huelva (capital),8699.0,279.0,599.0,143837.0,193.969563,416.443613,50.0
Condado-Campiña,10455.0,191.0,501.0,156231.0,122.254866,320.678995,25.0
Sierra de Huelva-Andévalo Central,2849.0,55.0,147.0,67391.0,81.613272,218.130017,10.0
Moguer,1336.0,41.0,80.0,21867.0,187.497142,365.848082,10.0
Aljaraque,1145.0,33.0,83.0,21474.0,153.674211,386.513924,6.0
Valverde del Camino,620.0,31.0,77.0,12750.0,243.137255,603.921569,4.0
Almonte,1859.0,19.0,99.0,24507.0,77.528869,403.966214,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,12.0,4.0,4.0,299.0,1337.792642,1337.792642,3.0
Villarrasa,180.0,7.0,14.0,2211.0,316.598824,633.197648,1.0
Valverde del Camino,620.0,31.0,77.0,12750.0,243.137255,603.921569,4.0
Niebla,241.0,3.0,21.0,4116.0,72.886297,510.204082,1.0
San Juan del Puerto,534.0,22.0,46.0,9411.0,233.768994,488.789714,2.0
Palos de la Frontera,690.0,19.0,56.0,11742.0,161.812298,476.920456,2.0
Huelva (capital),8699.0,279.0,599.0,143837.0,193.969563,416.443613,50.0
Almonte,1859.0,19.0,99.0,24507.0,77.528869,403.966214,3.0
Puebla de Guzmán,125.0,6.0,12.0,3092.0,194.049159,388.098318,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,558.0,1.0,14.0,7939.0,12.596045,176.344628,0.0,0.071429
Niebla,241.0,3.0,21.0,4116.0,72.886297,510.204082,1.0,0.142857
Almonte,1859.0,19.0,99.0,24507.0,77.528869,403.966214,3.0,0.191919
Zalamea la Real,94.0,1.0,4.0,3054.0,32.743942,130.975769,0.0,0.250000
Gibraleón,638.0,6.0,23.0,12737.0,47.106854,180.576274,2.0,0.260870
Cartaya,1357.0,10.0,36.0,20083.0,49.793358,179.256087,1.0,0.277778
Palma del Condado (La),1166.0,8.0,28.0,10801.0,74.067216,259.235256,0.0,0.285714
San Bartolomé de la Torre,302.0,3.0,9.0,3761.0,79.766020,239.298059,1.0,0.333333
Ayamonte,1323.0,3.0,9.0,21104.0,14.215315,42.645944,1.0,0.333333
